# The JuMP ecosystem for mathematical optimization: Topics

This notebook is part of a workshop at [JuliaCon 2018](http://juliacon.org/2018/) and is based on materials and notebooks from various sources including the [JuliaOpt notebooks](https://github.com/JuliaOpt/juliaopt-notebooks), the [2018 ISCO Spring School](https://github.com/joehuchette/ISCO-spring-school) and the [second annual JuMP-dev workshop](http://www.juliaopt.org/meetings/bordeaux2018/).

## Disclaimer
This notebook is only working under the versions:

- JuMP 0.19 (unreleased, but currently in master)

- MathOptInterface 0.4.1

- GLPK 0.6.0

In [2]:
using JuMP  
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver

## Duality

I can get duals from constraint references.

In [3]:
model = Model(optimizer = GLPK.GLPKOptimizerLP())
@variable(model, x >= 0)
@variable(model, y >= 0)
@constraint(model,inequality, x + y <= 1)         
@objective(model, Max, x + 2y)
JuMP.optimize(model)
@show JuMP.hasresultvalues(model)
@show JuMP.terminationstatus(model) == MOI.Success
@show JuMP.primalstatus(model) == MOI.FeasiblePoint
@show JuMP.dualstatus(model) == MOI.FeasiblePoint
@show JuMP.resultdual(inequality)

JuMP.hasresultvalues(model) = true
JuMP.terminationstatus(model) == MOI.Success = true
JuMP.primalstatus(model) == MOI.FeasiblePoint = true
JuMP.dualstatus(model) == MOI.FeasiblePoint = true


-2.0

JuMP.resultdual(inequality) = -2.0


I can also get "duals" for variable bounds.

In [4]:
@show JuMP.resultdual(JuMP.LowerBoundRef(x))
@show JuMP.resultdual(JuMP.LowerBoundRef(y))

0.0

JuMP.resultdual(JuMP.LowerBoundRef(x)) = 1.0
JuMP.resultdual(JuMP.LowerBoundRef(y)) = 0.0


For sign conventions and precise definition of "dual" problem see [Duality](http://www.juliaopt.org/MathOptInterface.jl/stable/apimanual.html#Duals-1) in [MOI Manual](http://www.juliaopt.org/MathOptInterface.jl/stable)

## JuMP Containers

In [1]:
using JuMP, GLPK, Compat

In [69]:
m = Model(optimizer = GLPK.GLPKOptimizerLP())
@variable(m, x[i=1:5, j=1:5])                        # Array
set_1 = Base.OneTo(5)
@variable(m, y[i=set_1, j=1:5])                      # Array
set_2 = 1:5
@variable(m, z1[i=1:5, j=set_2])                     # JuMPArray
@variable(m, z2[i=1:5, j=set_2], container = Array)
a = 1
@variable(m, zz1[i=1:5, j=a:5])                      # JuMPArray
@variable(m, zz2[i=1:5, j=a:5], container = Array)   # JuMPArray
set_3 = [:a, :b, :c]
@variable(m, w[i=set_2, j=set_3])                    # JuMPArray
@variable(m, t[i=set_2, j=1:i])                      # Dict
@variable(m, h[i = 1:5; j=isodd(i)])                 # Dict

Dict{Any,JuMP.VariableRef} with 3 entries:
  3 => h[3]
  5 => h[5]
  1 => h[1]

In [68]:
typeof(zz2)

Array{JuMP.VariableRef,2}

### `x[i=1:5, j=1:5]`

In [76]:
#@macroexpand @variable(m, x[i=1:5, j=1:5])  # Array
refcall, idxvars, idxsets, condition = JuMP.buildrefsets(:(x[i=1:5, j=1:5]))
containercode, autoduplicatecheck = JuMP.generatecontainer(VariableRef, idxvars, idxsets, :Auto)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

:((Main.Array){JuMP.VariableRef}(Main.undef, ((Main.length)(1:5), (Main.length)(1:5))...))

### `y[i=set_1, j=1:5], set_1 = Base.OneTo(5)`

In [77]:
#@macroexpand @variable(m, y[i=set_1, j=1:5]) # Array
refcall, idxvars, idxsets, condition = JuMP.buildrefsets(:(y[i=set_1, j=1:5]))
containercode, autoduplicatecheck = JuMP.generatecontainer(VariableRef, idxvars, idxsets, :Auto)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

:(if $(Expr(:&&, :(set_1 isa Main.Base.OneTo))) # /Users/jvielma/.julia/v0.6/JuMP/src/containers.jl, line 110:
        (Main.Array){JuMP.VariableRef}(Main.undef, ((Main.length)(set_1), (Main.length)(1:5))...)
    else  # /Users/jvielma/.julia/v0.6/JuMP/src/containers.jl, line 110:
        Main.JuMP.JuMPArray((Main.Array){JuMP.VariableRef}(Main.undef, ((Main.length)(set_1), (Main.length)(1:5))...), set_1, 1:5)
    end)

### `z[i=1:5, j=set_2], set_2 = 1:5`

In [70]:
#@macroexpand @variable(m, z1[i=1:5, j=set_2])  # JuMPArray
refcall, idxvars, idxsets, condition = JuMP.buildrefsets(:(z[i=1:5, j=set_2]))
containercode, autoduplicatecheck = JuMP.generatecontainer(VariableRef, idxvars, idxsets, :Auto)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

:(if $(Expr(:&&, :(set_2 isa Main.Base.OneTo))) # /Users/jvielma/.julia/v0.6/JuMP/src/containers.jl, line 110:
        (Main.Array){JuMP.VariableRef}(Main.undef, ((Main.length)(1:5), (Main.length)(set_2))...)
    else  # /Users/jvielma/.julia/v0.6/JuMP/src/containers.jl, line 110:
        Main.JuMP.JuMPArray((Main.Array){JuMP.VariableRef}(Main.undef, ((Main.length)(1:5), (Main.length)(set_2))...), 1:5, set_2)
    end)

In [71]:
#@macroexpand @variable(m, z2[i=1:5, j=set_2], container = Array) # Array  
refcall, idxvars, idxsets, condition = JuMP.buildrefsets(:(z[i=1:5, j=set_2]))
containercode, autoduplicatecheck = JuMP.generatecontainer(VariableRef, idxvars, idxsets, :Array)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

quote 
    $(Expr(:&&, :(Main.JuMP.validarrayindexset(set_2)))) || (Main.error)("Index set for array is not one-based interval.")
    (Main.Array){JuMP.VariableRef}(Main.undef, ((Main.length)(1:5), (Main.length)(set_2))...)
end

In [72]:
@code_typed JuMP.validarrayindexset(set_2)

CodeInfo(:(begin 
        return ((Core.getfield)(r, :start)::Int64 === 1)::Bool
    end))=>Bool

In [81]:
methods(JuMP.validarrayindexset)

# 3 methods for generic function "validarrayindexset":
validarrayindexset(::Base.OneTo) in JuMP at /Users/jvielma/.julia/v0.6/JuMP/src/containers.jl:8
validarrayindexset(r::UnitRange) in JuMP at /Users/jvielma/.julia/v0.6/JuMP/src/containers.jl:9
validarrayindexset(s) in JuMP at /Users/jvielma/.julia/v0.6/JuMP/src/containers.jl:10